In [ ]:
from zcash_client import ZcashClient, CONF_PATH

SAMPLES_BANDWIDTH_HEIGHT = "experiments/samples_height.csv"
SAMPLES_ATTACK_HEIGHT = "experiments/samples_attack_height.csv"
SAMPLES_BANDWIDTH_DIFFICULTY = "experiments/samples_difficulty.csv"
SAMPLES_ATTACK_DIFFICULTY = "experiments/samples_attack_difficulty.csv"

SAMPLES_BANDWIDTH_HEIGHT_NI = "experiments/samples_height_ni.csv"
SAMPLES_ATTACK_HEIGHT_NI = "experiments/samples_attack_height_ni.csv"
SAMPLES_BANDWIDTH_DIFFICULTY_NI = "experiments/samples_difficulty_ni.csv"
SAMPLES_ATTACK_DIFFICULTY_NI = "experiments/samples_attack_difficulty_ni.csv"

NA_VALUES_DATASET = "experiments/na_values.csv"
DATASET = "experiments/proof_size_dataset.csv"
COMPRESSION_DATASET = "experiments/compression_dataset.csv"
PLOT_PATH = "experiments/plots/"

ITERATIONS = 30

# ACTIVATION_HEIGHT = 903000
START_HEIGHT = 904000
CHAINTIP = 3154000
STEP = 25000

# Zcash hourly number of blocks mined
Dh = 48
# Estimated cost of a 51% attack on Zcash per hour
C51h = 2624 # USD

C_VALUES = [0.2, 0.35, 0.5, 0.65, 0.8]
L_VALUES = [100, Dh * 3, Dh * 4, Dh * 5]

# Generate test sets for each chaintip height value
client = ZcashClient.from_conf(CONF_PATH)

sample_cols = ['chaintip', 'c', 'L', 'samples']
diff_cols = ['height', 'total_work']

In [ ]:
import scipy.stats
import numpy as np
import pandas as pd

def df_stats(df: pd.DataFrame, group_key: str, data_key: str, confidence=0.95) -> pd.DataFrame:
    cidf = df.groupby([group_key])[data_key].describe()[['count', 'mean', 'std']].reset_index()
    cv = scipy.stats.t.ppf((1 + confidence) / 2., cidf['count']-1)
    cidf["lower_ci"] = cidf["mean"] - cv*(cidf["std"]/np.sqrt(cidf["count"]))
    cidf["upper_ci"] = cidf["mean"] + cv*(cidf["std"]/np.sqrt(cidf["count"]))
    cidf["ci_interval"] = cidf["upper_ci"] - cidf["lower_ci"]
    return cidf

In [ ]:
import matplotlib.pyplot as plt
from run_experiments import *
from parameter_optimizer import *
from matplotlib.ticker import FormatStrFormatter

SCALING_FACTOR = 2**20
HEADER_SIZE = 4 + 32 + 32 + 32 + 4 + 4 + 32 + 3 + 1344

ACTIVATION_HEIGHTS_MAINNET = {
    "heartwood": 903000,
    "canopy": 1046400,
    "nu5": 1687104,
    "nu6": 2726400
}

ACTIVATION_HEIGHTS_TESTNET = {
    "heartwood": 903800,
    "canopy": 1028500,
    "nu5": 1842420,
    "nu6": 2976000
}

df = pd.read_csv(NA_VALUES_DATASET)

plt.figure(figsize=(6, 4))
plt.rcParams['text.usetex'] = True
df["chaintip"] = df["chaintip"] / 1000000

# fmt=[marker][line][color]
plt.plot("chaintip", "c", "o-C0", data=df.query("protocol=='interactive' and N_a==25.0").iloc[::8], label=r'interactive, $N_a = 25$', ms=4)
plt.plot("chaintip", "c", "^-C1", data=df.query("protocol=='interactive' and N_a==50.0").iloc[::8], label=r'interactive, $N_a = 50$', ms=4)
plt.plot("chaintip", "c", "s-C2", data=df.query("protocol=='interactive' and N_a==100.0").iloc[::8], label=r'interactive, $N_a = 100$', ms=4)
plt.plot("chaintip", "c", "x-C3", data=df.query("protocol=='interactive' and N_a==500.0").iloc[::8], label=r'interactive, $N_a = 500$', ms=4)
plt.plot("chaintip", "c", "o--C0", data=df.query("protocol=='non_interactive' and N_a==25.0").iloc[::8], label=r'non-interactive, $N_a = 25$', ms=4)
plt.plot("chaintip", "c", "^--C1", data=df.query("protocol=='non_interactive' and N_a==50.0").iloc[::8], label=r'non-interactive, $N_a = 50$', ms=4)
plt.plot("chaintip", "c", "s--C2", data=df.query("protocol=='non_interactive' and N_a==100.0").iloc[::8], label=r'non-interactive, $N_a = 100$', ms=4)
plt.plot("chaintip", "c", "x--C3", data=df.query("protocol=='non_interactive' and N_a==500.0").iloc[::8], label=r'non-interactive, $N_a = 500$', ms=4)

plt.xlabel("Chain length (million blocks)")
plt.ylabel(r"$c^*$")
#plt.title("Optimal values of c by chosen attacker model and chain length")

ax = plt.gca()
# ax.locator_params(nbins=10, axis='x')
ax.locator_params(nbins=10, axis='y')
ax.set_xticks(df.query("protocol=='non_interactive' and N_a==25.0").iloc[::8]["chaintip"])
ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout()
plt.legend(bbox_to_anchor=(1, 1), loc='upper right', framealpha=0.5)
plt.grid(axis="y")
plt.savefig(PLOT_PATH + "c_opt.pdf")

In [ ]:
df = pd.read_csv(NA_VALUES_DATASET)

plt.figure(figsize=(6, 4))
plt.rcParams['text.usetex'] = True
df["chaintip"] = df["chaintip"] / 1000000

# fmt=[marker][line][color]
plt.plot("chaintip", "c", "o-C0", data=df.query("protocol=='interactive' and N_a==25.0").iloc[::8], label=r'\noindent interactive,\\$N_a = 25$', ms=4)
plt.plot("chaintip", "c", "^-C1", data=df.query("protocol=='interactive' and N_a==50.0").iloc[::8], label=r'\noindent interactive,\\$N_a = 50$', ms=4)
plt.plot("chaintip", "c", "s-C2", data=df.query("protocol=='interactive' and N_a==100.0").iloc[::8], label=r'\noindent interactive,\\$N_a = 100$', ms=4)
plt.plot("chaintip", "c", "x-C3", data=df.query("protocol=='interactive' and N_a==500.0").iloc[::8], label=r'\noindent interactive,\\$N_a = 500$', ms=4)
plt.plot("chaintip", "c", "o--C0", data=df.query("protocol=='non_interactive' and N_a==25.0").iloc[::8], label=r'\noindent non-interactive,\\$N_a = 25$', ms=4)
plt.plot("chaintip", "c", "^--C1", data=df.query("protocol=='non_interactive' and N_a==50.0").iloc[::8], label=r'\noindent non-interactive,\\$N_a = 50$', ms=4)
plt.plot("chaintip", "c", "s--C2", data=df.query("protocol=='non_interactive' and N_a==100.0").iloc[::8], label=r'\noindent non-interactive,\\$N_a = 100$', ms=4)
plt.plot("chaintip", "c", "x--C3", data=df.query("protocol=='non_interactive' and N_a==500.0").iloc[::8], label=r'\noindent non-interactive,\\$N_a = 500$', ms=4)

plt.xlabel("Chain length (million blocks)")
plt.ylabel(r"$c^*$")
#plt.title("Optimal values of c by chosen attacker model and chain length")

ax = plt.gca()
# ax.locator_params(nbins=10, axis='x')
ax.locator_params(nbins=10, axis='y')
ax.set_xticks(df.query("protocol=='non_interactive' and N_a==25.0").iloc[::8]["chaintip"])
ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout()
plt.subplots_adjust(right=0.8)
plt.legend(bbox_to_anchor=(1.38, 1), loc='upper right')
plt.grid(axis="y")
plt.savefig(PLOT_PATH + "c_opt_alt.pdf", bbox_inches="tight")

In [ ]:
from dateutil import parser
import matplotlib.ticker as ticker

def format_seconds_to_hms(x, pos):
    hours = int(x // 3600)
    minutes = int((x % 3600) // 60)
    #seconds = int(x % 60)
    return f"{hours:02}:{minutes:02}"

# [db disk size, live data disk size]
db_size_before = [269.3, 268.4]
db_size_after = [269, 268.9]

# timestamps = [ 
#     parser.isoparse(t) for t in [
#         "2025-04-12T17:42:26.839077Z", 
#         "2025-04-12T19:14:42.767906Z", 
#         "2025-04-12T21:14:50.559037Z", 
#         "2025-04-12T23:45:39.764118Z", 
#         "2025-04-14T18:47:04.191614Z", 
#         "2025-04-15T04:43:33.471736Z",
#         "2025-04-15T08:05:19.101426Z"
#     ]
# ]

timestamps = [ 
    parser.isoparse(t) for t in [
        "2025-05-29T07:02:58.364888Z",
        "2025-05-29T08:41:46.010495Z",
        "2025-05-29T10:47:09.806158Z",
        "2025-05-29T13:22:47.045034Z",
        "2025-05-30T16:24:04.835977Z",
        "2025-05-30T20:15:31.180192Z",
        "2025-05-30T22:49:32.019318Z"
    ]
]

db_timestamps = [ 
    parser.isoparse(t) for t in [
        "2025-05-31T09:42:35.597944Z", 
        "2025-05-31T09:43:13.493703Z", 
        "2025-05-31T09:47:58.264654Z", 
        "2025-05-31T13:08:09.070503Z", 
        "2025-05-31T13:26:11.908590Z",
        "2025-05-31T13:30:53.319793Z"
    ]
]

verify_timestamps = [ 
    parser.isoparse(t) for t in [
        "2025-05-31T13:30:53.319822Z", 
        "2025-05-31T13:32:02.187191Z", 
        "2025-05-31T13:37:06.205567Z", 
        "2025-05-31T19:47:07.857120Z", 
        "2025-05-31T19:51:54.342540Z", 
        "2025-05-31T19:53:13.971202Z"
    ]
]

new_timestamps = [ 
    parser.isoparse(t) for t in [
        "2025-05-25T13:40:53.824740Z", 
        "2025-05-25T15:18:10.188536Z", 
        "2025-05-25T17:25:17.486057Z", 
        "2025-05-25T20:18:57.448569Z", 
        "2025-05-26T23:13:08.831277Z", 
        "2025-05-27T03:02:16.977734Z",
        "2025-05-27T05:33:27.581280Z"
    ]
]

start = timestamps[0]
db_start = db_timestamps[0]
verify_start = verify_timestamps[0]
new_start = new_timestamps[0]

deltas = [(t - start).total_seconds() for t in timestamps]
db_deltas = [((t - db_start).total_seconds() + (v - verify_start).total_seconds()) for (t, v) in zip(db_timestamps, verify_timestamps)]
new_deltas = [(t - new_start).total_seconds() for t in new_timestamps]

block_count = [0, 500_000, 1_000_000, 1_500_000, 2_000_000, 2_500_000, 2_942_449]
db_block_count = [903_000, 1_000_000, 1_500_000, 2_000_000, 2_500_000, 2_942_861]
new_block_count = [0, 500_000, 1_000_000, 1_500_000, 2_000_000, 2_500_000, 2_938_181]

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(block_count, deltas, marker='o', linestyle='-', label="full sync (old format)")
ax.plot(new_block_count, new_deltas, marker='x', linestyle='--', label="full sync (new format)")
ax.plot(db_block_count, db_deltas, marker='s', linestyle='-.', label="database format upgrade")
ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(format_seconds_to_hms))
ax.yaxis.set_major_locator(ticker.MultipleLocator(10800))
ax.legend()
ax.grid(axis='y')
plt.xlabel("chain length (blocks)")
plt.ylabel("time elapsed")
plt.tight_layout()
plt.savefig(PLOT_PATH + 'db_upgrade_time.pdf')

In [ ]:
df = pd.read_csv(DATASET)
scaled_df = df.copy()
scaled_df['proof_size'] /= SCALING_FACTOR
scaled_df['chaintip'] /= 1000000
sampled_df = scaled_df.query("N_a == 50.0 & protocol=='interactive' & compression==False")

plt.figure(figsize=(6, 4))

# plt.plot("chaintip", "proof_size", "o-C0", data=sampled_df.query("optimization_type=='none' and difficulty=='constant'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='Unoptimized,\n constant difficulty', ms=4)
plt.plot("chaintip", "proof_size", "s-C0", data=sampled_df.query("optimization_type=='cache' and difficulty=='variable'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index().iloc[::4], label='verifier', ms=4)
plt.plot("chaintip", "proof_size", "x--C1", data=sampled_df.query("optimization_type=='cache' and difficulty=='constant'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index().iloc[::4], label='fixed-difficulty verifier')
plt.plot("chaintip", "proof_size", ".-C4", data=sampled_df.query("optimization_type=='none' and difficulty=='variable'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='non-cached verifier')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['heartwood'] / 1000000, color='green', linestyle='-')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['canopy'] / 1000000, color='green', linestyle='--')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu5'] / 1000000, color='green', linestyle='-.')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu6'] / 1000000, color='green', linestyle = ':')
plt.text(0.91, 1.3, "Heartwood")
plt.text(1.05, 0.85, "Canopy")
plt.text(1.7, 1, "NU5")
plt.text(2.74, 1, "NU6")

plt.xlabel("chain length (million blocks)")
plt.ylabel("proof size (MiB)")
#plt.title(r"Proof size by chain length, interactive ($N_a = 50$)")

ax = plt.gca()
# ax.set_xlim(0, CHAINTIP)
# ax.locator_params(nbins=20, axis='x')
# ax.locator_params(nbins=20, axis='y')
ax.set_xticks(sampled_df["chaintip"].iloc[::8].unique())
ax.set_yticks([i/10 for i in range(6, 36, 2)])
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout()
# plt.subplots_adjust(right=0.8)
plt.legend(loc='center', bbox_to_anchor=(0.75, 0.45))
# plt.legend(loc='upper right', bbox_to_anchor=(1.4, 1))
plt.grid(axis="y")
plt.savefig(PLOT_PATH + "proof_size_alt.pdf", bbox_inches="tight")

In [ ]:
print(df_stats(sampled_df.query("optimization_type=='cache' and difficulty=='variable'"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='cache' and difficulty=='constant'"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='none' and difficulty=='variable'"), 'chaintip', 'proof_size').max()['ci_interval'])

In [ ]:
df = pd.read_csv(DATASET)
scaled_df = df.copy()
scaled_df['proof_size'] /= SCALING_FACTOR
scaled_df['chaintip'] /= 1000000
sampled_df = scaled_df.query("N_a == 50.0 & protocol=='non_interactive' & compression==False")

plt.figure(figsize=(6, 4))

plt.plot("chaintip", "proof_size", "s-C0", data=sampled_df.query("optimization_type=='cache' and difficulty=='variable'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index().iloc[::4], label='verifier', ms=4)
plt.plot("chaintip", "proof_size", "x--C1", data=sampled_df.query("optimization_type=='cache' and difficulty=='constant'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index().iloc[::4], label='fixed-difficulty verifier')
plt.plot("chaintip", "proof_size", ".-C4", data=sampled_df.query("optimization_type=='none' and difficulty=='variable'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='non-cached verifier')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['heartwood'] / 1000000, color='green', linestyle='-')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['canopy'] / 1000000, color='green', linestyle='--')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu5'] / 1000000, color='green', linestyle='-.')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu6'] / 1000000, color='green', linestyle = ':')
plt.text(0.91, 1.4, "Heartwood")
plt.text(1.05, 1, "Canopy")
plt.text(1.7, 1, "NU5")
plt.text(2.74, 1, "NU6")

plt.xlabel("chain length (million blocks)")
plt.ylabel("proof size (MiB)")
#plt.title(r"Proof size by chain length, interactive ($N_a = 50$)")

ax = plt.gca()
# ax.set_xlim(0, CHAINTIP)
# ax.locator_params(nbins=20, axis='x')
# ax.locator_params(nbins=20, axis='y')
ax.set_yticks([i/10 for i in range(6, 36, 2)])
ax.set_xticks(sampled_df["chaintip"].iloc[::8].unique())
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout()
# plt.subplots_adjust(right=0.8)
plt.legend(loc='center', framealpha=0.6, bbox_to_anchor=(0.75, 0.55))
# plt.legend(loc='upper right', bbox_to_anchor=(1.40, 1))
plt.grid(axis="y")
plt.savefig(PLOT_PATH + "proof_size_alt_ni.pdf", bbox_inches="tight")

In [ ]:
print(df_stats(sampled_df.query("optimization_type=='cache' and difficulty=='variable'"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='cache' and difficulty=='constant'"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='none' and difficulty=='variable'"), 'chaintip', 'proof_size').max()['ci_interval'])

In [ ]:
df = pd.read_csv(DATASET)
scaled_df = df.copy()
scaled_df['proof_size'] /= SCALING_FACTOR
scaled_df['chaintip'] /= 1000000
sampled_df = scaled_df.query("N_a == 50.0 & protocol=='interactive' & compression==False")
sampled_df_ni = scaled_df.query("N_a == 50.0 & protocol=='non_interactive' & compression==False")

plt.figure(figsize=(6, 4))

plt.plot("chaintip", "proof_size", "o-C0", data=sampled_df.query("optimization_type=='cache' and difficulty=='variable'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index().iloc[::4], label='interactive verifier', ms=4)
plt.plot("chaintip", "proof_size", "x--C1", data=sampled_df.query("optimization_type=='cache' and difficulty=='constant'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index().iloc[::4], label='fixed-difficulty\ninteractive verifier')
plt.plot("chaintip", "proof_size", ".-C4", data=sampled_df.query("optimization_type=='none' and difficulty=='variable'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='non-cached\ninteractive verifier', ms=4)

plt.plot("chaintip", "proof_size", "s-.C2", data=sampled_df_ni.query("optimization_type=='cache' and difficulty=='variable'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index().iloc[::4], label='non-interactive\nverifier', ms=4)
plt.plot("chaintip", "proof_size", "d:C3", data=sampled_df_ni.query("optimization_type=='cache' and difficulty=='constant'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index().iloc[::4], label='non-interactive\nfixed-difficulty\nverifier', ms=4)
plt.plot("chaintip", "proof_size", ".--C5", data=sampled_df_ni.query("optimization_type=='none' and difficulty=='variable'").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='non-interactive\nnon-cached verifier')

plt.axvline(ACTIVATION_HEIGHTS_MAINNET['heartwood'] / 1000000, color='C7', linestyle='-')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['canopy'] / 1000000, color='C7', linestyle='--')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu5'] / 1000000, color='C7', linestyle='-.')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu6'] / 1000000, color='C7', linestyle = ':')

plt.text(0.92, 3.4, "Heartwood")
plt.text(1.05, 3.2, "Canopy")
plt.text(1.7, 3.4, "NU5")
plt.text(2.74, 3.4, "NU6")

plt.xlabel("chain length (million blocks)")
plt.ylabel("proof size (MiB)")
#plt.title(r"Proof size by chain length, interactive ($N_a = 50$)")

ax = plt.gca()
# ax.set_xlim(0, CHAINTIP)
# ax.locator_params(nbins=20, axis='x')
ax.locator_params(nbins=20, axis='y')
ax.set_xticks(sampled_df["chaintip"].iloc[::8].unique())
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout()
plt.subplots_adjust(right=0.8)
# plt.legend(loc='center', framealpha=0.6, bbox_to_anchor=(0.75, 0.55), alignment='right', markerfirst=False)
plt.legend(loc='upper right', bbox_to_anchor=(1.40, 1.02))
plt.grid(axis="y")
plt.savefig(PLOT_PATH + "proof_size_comb.pdf", bbox_inches="tight")

In [ ]:
df = pd.read_csv(DATASET)
scaled_df = df.copy()
scaled_df['proof_size'] /= SCALING_FACTOR
scaled_df['chaintip'] /= 1000000
sampled_df = scaled_df.query("N_a == 50.0 & protocol=='interactive' & difficulty=='variable'").iloc[::4]

plt.figure(figsize=(6, 4))

plt.plot("chaintip", "proof_size", "o-C0", data=sampled_df.query("optimization_type=='cache' & compression==False").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='(unoptimized)\nverifier', ms=4)
plt.plot("chaintip", "proof_size", "x-C1", data=sampled_df.query("optimization_type=='cache' & compression==True").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='zipped-proof\nverifier')
plt.plot("chaintip", "proof_size", "s-C2", data=sampled_df.query("optimization_type=='aggregate' & compression==True").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='aggregate-proof\nverifier', ms=4)
plt.plot("chaintip", "proof_size", "d-C3", data=sampled_df.query("optimization_type=='flyclient_friendly' & compression==True").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='flyclient-friendly\npow verifier', ms=5)
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['heartwood'] / 1000000, color='C7', linestyle='-')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['canopy'] / 1000000, color='C7', linestyle='--')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu5'] / 1000000, color='C7', linestyle='-.')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu6'] / 1000000, color='C7', linestyle = ':')

plt.text(0.92, 1.41, "Heartwood")
plt.text(1.05, 1.51, "Canopy")
plt.text(1.7, 1.51, "NU5")
plt.text(2.74, 1.51, "NU6")

plt.xlabel("chain length (million blocks)")
plt.ylabel("proof size (MiB)")
#plt.title(r"Proof size by chain length, interactive ($N_a = 50$)")

ax = plt.gca()
# ax.set_xlim(0, CHAINTIP)
# ax.locator_params(nbins=20, axis='x')
ax.locator_params(nbins=20, axis='y')
ax.set_xticks(sampled_df["chaintip"].iloc[::8].unique())
ax.set_yticks([i/10 for i in range(0, 17)])
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout()
plt.subplots_adjust(right=0.8)
# plt.legend(loc='center', framealpha=0.6, bbox_to_anchor=(0.75, 0.55), alignment='right', markerfirst=False)
plt.legend(loc='upper right', bbox_to_anchor=(1.40, 1.02))
plt.grid(axis="y")
plt.savefig(PLOT_PATH + "opt_proof_size.pdf", bbox_inches="tight")

In [ ]:
print(df_stats(sampled_df.query("optimization_type=='cache' & compression==False"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='cache' & compression==True"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='aggregate' & compression==True"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='flyclient_friendly' & compression==True"), 'chaintip', 'proof_size').max()['ci_interval'])

In [ ]:
df = pd.read_csv(DATASET)
scaled_df = df.copy()
scaled_df['proof_size'] /= SCALING_FACTOR
scaled_df['chaintip'] /= 1000000
sampled_df = scaled_df.query("N_a == 50.0 & protocol=='non_interactive' & difficulty=='variable'").iloc[::4]

plt.figure(figsize=(6, 4))

plt.plot("chaintip", "proof_size", "o-C0", data=sampled_df.query("optimization_type=='cache' & compression==False").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='(unoptimized)\nverifier', ms=4)
plt.plot("chaintip", "proof_size", "x-C1", data=sampled_df.query("optimization_type=='cache' & compression==True").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='zipped-proof\nverifier')
plt.plot("chaintip", "proof_size", "s-C2", data=sampled_df.query("optimization_type=='aggregate' & compression==True").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='aggregate-proof\nverifier', ms=4)
plt.plot("chaintip", "proof_size", "d-C3", data=sampled_df.query("optimization_type=='flyclient_friendly' & compression==True").groupby("chaintip")["proof_size"].mean().to_frame().reset_index(), label='flyclient-friendly\npow verifier', ms=5)
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['heartwood'] / 1000000, color='C7', linestyle='-')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['canopy'] / 1000000, color='C7', linestyle='--')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu5'] / 1000000, color='C7', linestyle='-.')
plt.axvline(ACTIVATION_HEIGHTS_MAINNET['nu6'] / 1000000, color='C7', linestyle = ':')

plt.text(0.92, 1.41, "Heartwood")
plt.text(1.05, 1.5, "Canopy")
plt.text(1.7, 1.5, "NU5")
plt.text(2.74, 1.43, "NU6")

plt.xlabel("chain length (million blocks)")
plt.ylabel("proof size (MiB)")
#plt.title(r"Proof size by chain length, interactive ($N_a = 50$)")

ax = plt.gca()
# ax.set_xlim(0, CHAINTIP)
# ax.locator_params(nbins=20, axis='x')
ax.locator_params(nbins=20, axis='y')
ax.set_xticks(sampled_df["chaintip"].iloc[::8].unique())
plt.gca().xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

plt.tight_layout()
plt.subplots_adjust(right=0.8)
# plt.legend(loc='center', framealpha=0.6, bbox_to_anchor=(0.75, 0.55), alignment='right', markerfirst=False)
plt.legend(loc='upper right', bbox_to_anchor=(1.40, 1.02))
plt.grid(axis="y")
plt.savefig(PLOT_PATH + "opt_proof_size_ni.pdf", bbox_inches="tight")

In [ ]:
print(df_stats(sampled_df.query("optimization_type=='cache' & compression==False"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='cache' & compression==True"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='aggregate' & compression==True"), 'chaintip', 'proof_size').max()['ci_interval'])
print(df_stats(sampled_df.query("optimization_type=='flyclient_friendly' & compression==True"), 'chaintip', 'proof_size').max()['ci_interval'])